# Progetto Lar Generators (Studio esecutivo)

Per avviare una istanza Jupyter con 8 threads, eseguire il seguente codice dal proprio terminale e selezionare il kernel appena creato.

```julia
using IJulia
IJulia.installkernel("Julia 8 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "8",
))
````

In [1]:
Threads.nthreads()

8

## Codice versione originale

In [7]:
using LinearAlgebraicRepresentation, SparseArrays, ViewerGL
Lar = LinearAlgebraicRepresentation;
GL = ViewerGL
using IntervalTrees, LinearAlgebra

function pointInPolygonClassification(V, EV)
    function pointInPolygonClassification0(pnt)
        x, y = pnt
        xmin, xmax, ymin, ymax = x, x, y, y
        tilecode = Lar.setTile([ymax, ymin, xmax, xmin])
        count, status = 0, 0

        for (k, edge) in enumerate(EV)
            p1, p2 = V[:, edge[1]], V[:, edge[2]]
            (x1, y1), (x2, y2) = p1, p2
            c1, c2 = tilecode(p1), tilecode(p2)
            c_edge, c_un, c_int = c1 ⊻ c2, c1 | c2, c1 & c2

            if (c_edge == 0) & (c_un == 0)
                return "p_on"
            elseif (c_edge == 12) & (c_un == c_edge)
                return "p_on"
            elseif c_edge == 3
                if c_int == 0
                    return "p_on"
                elseif c_int == 4
                    count += 1
                end
            elseif c_edge == 15
                x_int = ((y - y2) * (x1 - x2) / (y1 - y2)) + x2
                if x_int > x
                    count += 1
                elseif x_int == x
                    return "p_on"
                end
            elseif (c_edge == 13) & ((c1 == 4) | (c2 == 4))
                crossingTest(1, 2, status, count)
            elseif (c_edge == 14) & ((c1 == 4) | (c2 == 4))
                crossingTest(2, 1, status, count)
            elseif c_edge == 7
                count += 1
            elseif c_edge == 11
                count = count
            elseif c_edge == 1
                if c_int == 0
                    return "p_on"
                elseif c_int == 4
                    crossingTest(1, 2, status, count)
                end
            elseif c_edge == 2
                if c_int == 0
                    return "p_on"
                elseif c_int == 4
                    crossingTest(2, 1, status, count)
                end
            elseif (c_edge == 4) & (c_un == c_edge)
                return "p_on"
            elseif (c_edge == 8) & (c_un == c_edge)
                return "p_on"
            elseif c_edge == 5
                if (c1 == 0) | (c2 == 0)
                    return "p_on"
                else
                    crossingTest(1, 2, status, count)
                end
            elseif c_edge == 6
                if (c1 == 0) | (c2 == 0)
                    return "p_on"
                else
                    crossingTest(2, 1, status, count)
                end
            elseif (c_edge == 9) & ((c1 == 0) | (c2 == 0))
                return "p_on"
            elseif (c_edge == 10) & ((c1 == 0) | (c2 == 0))
                return "p_on"
            end
        end
        if (round(count) % 2) == 1
            return "p_in"
        else
            return "p_out"
        end
    end
    return pointInPolygonClassification0
end

function settestpoints2d(W, f, copEV, copFE)
    e = findnz(copFE[f, :])[1][1]
    v1, v2 = findnz(copEV[e, :])[1]
    t = W[v2, :] - W[v1, :]
    n = [-t[2], t[1]]
    p0 = (W[v1, :] + W[v2, :]) ./ 2
    ϵ = 1.0e-4
    ptest1 = p0 + ϵ * n
    ptest2 = p0 - ϵ * n
    return ptest1, ptest2
end


function getinternalpoint2d(W, f, copEV, copFE)
    ptest1, ptest2 = settestpoints2d(W, f, copEV, copFE)
    edges = [findnz(copEV[e, :])[1] for e in findnz(copFE[f, :])[1]]
    V = convert(Lar.Points, W')
    classify = pointInPolygonClassification(V, edges)
    if classify(ptest1) == "p_in"
        return ptest1
    elseif classify(ptest2) == "p_in"
        return ptest2
    else
        error("classifying inner point in face $f")
    end
end

function chainbasis2polygons(copEV, copFE)
    FE = [findnz(copFE[k, :])[1] for k = 1:copFE.m]
    EV = [findnz(copEV[k, :])[1] for k = 1:copEV.m]

    FEs = []
    EVs = []
    FVs = []

    for f = 1:copFE.m
        push!(FEs, collect(Set(cat([e for e in FE[f]]; dims=1))))
        push!(EVs, [EV[e] for e in FE[f]])
        push!(FVs, collect(Set(cat([EV[e] for e in FE[f]]; dims=1))))
    end
    polygons = collect(zip(EVs, FVs, FEs))
    return polygons, FE
end


function internalpoints2d(W, copEV, copFE)
    pols, FE = chainbasis2polygons(copEV, copFE)
    internalpoints = []
    for f = 1:length(pols)
        (EV, FV, FE) = pols[f]
        internalpoint = getinternalpoint2d(W, f, copEV, copFE)
        push!(internalpoints, internalpoint)
    end
    return internalpoints
end


function testinternalpoint2d(listOfModels)
    function testinternalpoint0(testpoint)
        intersectedfaces = []
        for (k, model) in enumerate(listOfModels)
            verts, edges = model
            classify = pointInPolygonClassification(verts, edges)
            inOut = classify(testpoint)
            if inOut == "p_in"
                push!(intersectedfaces, k)
            end
        end
        return intersectedfaces
    end
    return testinternalpoint0
end

function bool2d(assembly)
    V, EV = Lar.struct2lar(assembly)
    cop_EW = convert(Lar.ChainOp, Lar.coboundary_0(EV::Lar.Cells))
    W = convert(Lar.Points, V')
    W, copEV, copFE = Lar.Arrangement.planar_arrangement(W::Lar.Points, cop_EW::Lar.ChainOp)
    innerpoints = internalpoints2d(W, copEV, copFE[1:end, :])
    listOfModels = Lar.evalStruct(assembly)
    boolmatrix = BitArray(undef, length(innerpoints), length(listOfModels))
    containmenttest = testinternalpoint2d(listOfModels)
    for (k, point) in enumerate(innerpoints)
        cells = containmenttest(point)
        for l in cells
            boolmatrix[k, l] = 1
        end
    end
    return W, copEV, copFE, boolmatrix
end

bool2d (generic function with 1 method)

## Esempio versione originale

In [8]:
using ViewerGL, SparseArrays
import LinearAlgebraicRepresentation as Lar
GL = ViewerGL

T=3
n,m = 1,1
V,(VV,EV,FV) = Lar.cuboidGrid([n,m],true)
square = V,EV
q = sqrt(2)
assembly = Lar.Struct([
    Lar.Struct([ Lar.t( rand(0:0.1:0.5), rand(0:0.1:0.5) ), Lar.r(rand(0:0.1:1)), square ]) for i in 1:T
])

V,EV = Lar.struct2lar(assembly)
GL.VIEW([ GL.GLGrid(V,EV, GL.COLORS[1],1), GL.GLFrame2 ]);

W, copEV, copFE, boolmatrix = bool2d(assembly)

A = boolmatrix[:,1]
B = boolmatrix[:,2]
C = boolmatrix[:,3]

AorB = A .| B .| C
AandB = A .& B .& C
AxorB = A .⊻ B .⊻ C

union = Matrix(copFE)' * Int.(AorB)
intersection = Matrix(copFE)' * Int.(AandB)
xor = Matrix(copFE)' * AxorB

V = convert(Lar.Points,W')
EV = Lar.cop2lar(copEV)
EVor = [ev for (k,ev) in enumerate(EV) if abs(union[k])==1 ]
EVand = [ev for (k,ev) in enumerate(EV) if abs(intersection[k])==1 ]
EVxor = [ev for (k,ev) in enumerate(EV) if abs(xor[k])==1 ]
GL.VIEW([ GL.GLGrid(V,EVor, GL.COLORS[1],1), GL.GLFrame2 ]);
GL.VIEW([ GL.GLGrid(V,EVand, GL.COLORS[1],1), GL.GLFrame2 ]);
GL.VIEW([ GL.GLGrid(V,EVxor, GL.COLORS[1],1), GL.GLFrame2 ]);

model = (V,[VV, EVxor])
GL.VIEW( GL.numbering(.5)( model,GL.COLORS[1],0.1 ) );


## Perfomance versione originale

In [15]:
@elapsed bool2d(assembly)

0.00798625

## Esempio del codice sviluppato ed ottimizzato

In [1]:
using Pkg
Pkg.add(url="https://github.com/paolomazzitti/CPD22-9")

    Updating git-repo `https://github.com/paolomazzitti/CPD22-9`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [18]:
using CPD9, ViewerGL, SparseArrays
GL = ViewerGL

T=3
n,m = 1,1
V,(VV,EV,FV) = CPD9.cuboidGrid([n,m],true)
square = V,EV
q = sqrt(2)

assemblyCPD9 = CPD9.Struct([
    CPD9.Struct([ CPD9.t( rand(0:0.1:0.5), rand(0:0.1:0.5) ), CPD9.r(rand(0:0.1:1)), square ]) for i in 1:T
])

V,EV = CPD9.struct2lar(assemblyCPD9)
GL.VIEW([ GL.GLGrid(V,EV, GL.COLORS[1],1), GL.GLFrame2 ]);

W, copEV, copFE, boolmatrix = CPD9.bool2d(assembly)

A = boolmatrix[:,1]
B = boolmatrix[:,2]
C = boolmatrix[:,3]

AorB = A .| B .| C
AandB = A .& B .& C
AxorB = A .⊻ B .⊻ C

union = Matrix(copFE)' * Int.(AorB)
intersection = Matrix(copFE)' * Int.(AandB)
xor = Matrix(copFE)' * AxorB

V = convert(CPD9.Points,W')
EV = CPD9.cop2lar(copEV)
EVor = [ev for (k,ev) in enumerate(EV) if abs(union[k])==1 ]
EVand = [ev for (k,ev) in enumerate(EV) if abs(intersection[k])==1 ]
EVxor = [ev for (k,ev) in enumerate(EV) if abs(xor[k])==1 ]
GL.VIEW([ GL.GLGrid(V,EVor, GL.COLORS[1],1), GL.GLFrame2 ]);
GL.VIEW([ GL.GLGrid(V,EVand, GL.COLORS[1],1), GL.GLFrame2 ]);
GL.VIEW([ GL.GLGrid(V,EVxor, GL.COLORS[1],1), GL.GLFrame2 ]);

model = (V,[VV, EVxor])
GL.VIEW( GL.numbering(.5)( model,GL.COLORS[1],0.1 ) );


## Performance codice sviluppato e ottimizzato

In [19]:
@elapsed CPD9.bool2d(assemblyCPD9)

0.006448084